In [25]:
#設定讀取資料的年份與時段

y = 104     # '110'年或'104'年
t = '昏峰'    # '晨峰'或'昏峰'
case = '士林北投三重'    # '原始'或'士林北投三重'或'全區域'

In [26]:
#.xls to .csv file

import pandas as pd
import numpy as np
import os

if y == 110:
    input_file = 'TRTS4S_晨昏峰小時指派OD.xlsx'
if y == 104:
    input_file = 'TRTS4S_Y104晨昏峰小時指派OD_轉向_屏柵.xlsx'

output_file = f'{y}{t}_demand.csv'

if os.path.isfile(output_file):
    print(f"{output_file} 已存在，直接轉換為 dat 檔")
else:
    df = pd.read_excel(input_file, sheet_name=f'{y}年指派OD{t}小時')
    df.fillna(0, inplace=True)
    #save as csv
    df.to_csv(output_file, index=False, sep=',')
    print(f"Successfully saved {output_file}")

104昏峰_demand.csv 已存在，直接轉換為 dat 檔


## CODE

In [27]:
#.csv to .dat file

from tqdm import tqdm
import pandas as pd
import numpy as np

df = pd.read_csv(f'{y}{t}_demand.csv')
df.fillna(0, inplace=True)
rows = []

PCU = pd.read_csv('toNeihu_PCU.csv')
toNeihu_carPCU = PCU.iloc[:, 0].tolist()
toNeihu_motoPCU = PCU.iloc[:, 1].tolist()

#482-502
tamsui = [i for i in range(482,503)]

count = 0

# Modification for Beitou-Shihlin Technical Park (BSTP)
Nsum = 0
BSTP = [207,209,213]  
BeiShi = [i for i in [*range(1, 29), *range(187, 214), *range(482,503)]] #淡水北投士林
SanLu = [i for i in [*range(583, 594), *range(319, 326)]] #蘆洲三重(高速公路以北區域)
SanLu_all = [i for i in range(319, 340)]
Neihu = [146,148,149,150,154,155,156,157,158,159] #內湖科技園區
Taipei = [i for i in [*range(29, 131), *range(169, 187), *range(214, 268), *range(289, 304)]] #大同、大安、中山、中正、文山、松山、信義、萬華
Case1a_OD_SUM = 105643.80  # 人次
Case1b_OD_SUM = 230409.56  # 人次
Case2_OD_SUM = 3049025.25  # 人次

# 運具分配 [晨峰, 昏峰]
car_percent = [0.292, 0.3117]
moto_percent = [0.359, 0.3907]
bus_percent = [0.348, 0.2983]

# 乘載率 [晨峰, 昏峰]
car_load_rate = [1.698, 1.678]
moto_load_rate = [1.139, 1.209]
bus_load_rate = [19.0, 20.7]

# 當量
car_equiv = 1
moto_equiv = 0.3
bus_equiv = 1.8

# CASE: 
OBJ_transfer = 35000
Neihu_dest_sum = 7840
BSTP_transfer = 35000
BSTP_dest_sum = 2352
Neihu_carPCU_sum = 7087.399478126333
Neihu_motoPCU_sum = 5263.020866000018

# 人次
# 晨峰
toNeihu = 218885
# Case1a
Case1a_toNeihu = 17689
Case1a_toTaipei = 198709
# Case1b
Case1b_toNeihu = 25728
Case1b_toTaipei = 188238
# Case2
Case2_OD_SUM = 6063477
toBSTP = 19622

# 昏峰
fromNeihu = 162326
# Case1a
Case1a_fromNeihu = 7772
Case1a_fromTaipei = 177188
# Case1b
Case1b_fromNeihu = 13453
Case1b_fromTaipei = 187980
# Case2
Case2_OD_SUM = 6699120
fromBSTP = 11743

OTfactor = fromNeihu / toNeihu  # 加班係數

三蘆往台北 = 66878
三蘆往內湖 = 7368
淡北往台北 = 121360
淡北往內湖 = 17814

# Helper function for converting PCU
if t == '晨峰':
    tp = 0
elif t == '昏峰':
    tp = 1

# trips_sum = 0
# for index, row in tqdm(df.iterrows(), total=len(df)):
#     origin = row["I"]
#     dest = row["J"]
#     moto = row["MOTO"]
#     car = row["ALL"] - moto
#     bus = row["BUS"]
#     trips = car * car_equiv * car_load_rate[tp] + moto * moto_equiv * moto_load_rate[tp] + bus * bus_equiv * bus_load_rate[tp]
#     if (origin in BeiShi or origin in SanLu) and (dest in Neihu):
#         trips_sum += trips
# #  and (dest in BeiShi or dest in SanLu or dest in tamsui)
# print(f"{t}: {trips_sum}")

In [28]:
OTfactor = fromNeihu / toNeihu  # 加班係數
if t == '晨峰':
    OBJ_transfer = 35000 * (car_percent[0] + moto_percent[0] + bus_percent[0]) 
elif t == '昏峰':
    OBJ_transfer = 35000 * (car_percent[1] + moto_percent[1] + bus_percent[1]) * OTfactor


# PCU Conversion (time_period: 0 for '晨峰', 1 for '昏峰')
def convert_to_pcu(demand, veh_type, time_period):
    if veh_type == '1':  # Car
        return demand * car_percent[time_period] / car_load_rate[time_period] * car_equiv
    elif veh_type == '2':  # Motorcycle
        return demand * moto_percent[time_period] / moto_load_rate[time_period] * moto_equiv
    elif veh_type == '3':  # Bus
        return demand * bus_percent[time_period] / bus_load_rate[time_period] * bus_equiv
    else:
        return 0

def direct_convert_to_pcu(demand, veh_type, time_period):
    if veh_type == '1':  # Car
        return demand / car_load_rate[time_period] * car_equiv
    elif veh_type == '2':  # Motorcycle
        return demand / moto_load_rate[time_period] * moto_equiv
    elif veh_type == '3':  # Bus
        return demand / bus_load_rate[time_period] * bus_equiv
    else:
        return 0
    
# Inverse PCU Conversion (time_period: 0 for '晨峰', 1 for '昏峰')
def convert_to_demand(pcu, veh_type, time_period):
    if veh_type == '1':  # Car
        return pcu / car_equiv * car_load_rate[time_period] / car_percent[time_period]
    elif veh_type == '2':  # Motorcycle
        return pcu / moto_equiv * moto_load_rate[time_period] / moto_percent[time_period]
    elif veh_type == '3':  # Bus
        return pcu / bus_equiv * bus_load_rate[time_period] / bus_percent[time_period]
    else:
        return 0
    
def direct_convert_to_demand(pcu, veh_type, time_period):
    if veh_type == '1':  # Car
        return pcu * car_load_rate[time_period] / car_equiv
    elif veh_type == '2':  # Motorcycle
        return pcu * moto_load_rate[time_period] / moto_equiv
    elif veh_type == '3':  # Bus
        return pcu * bus_load_rate[time_period] / bus_equiv
    else:
        return 0

#use tqdm to show the progress for 614656 rows
print("demand.dat is generating...")
for index, row in tqdm(df.iterrows(), total=len(df)):
    origin = row["I"]
    dest = row["J"]
    moto = row["MOTO"]
    car = row["ALL"] - moto
    bus = row["BUS"]
    if t == '晨峰':
        car_demand = car + bus*1.8/19.0
    else:
        car_demand = car + bus*1.8/20.7

    out_of_cordon=[i for i in range(692,751)]+[783,784]
    if t == '晨峰':            
        if origin not in out_of_cordon and dest not in out_of_cordon:
            if case == '士林北投三重':
                if (origin in BeiShi or origin in SanLu or origin in tamsui) and dest in Taipei:
                    # 修正：先計算轉移量，再減少原始需求
                    car_transfer = car_demand * (0.5 * OBJ_transfer / Case1b_toTaipei)
                    moto_transfer = moto * (0.5 * OBJ_transfer / Case1b_toTaipei)
                    
                    car_demand = car_demand * (1 - (0.5 * OBJ_transfer / Case1b_toTaipei))
                    moto = moto * (1 - (0.5 * OBJ_transfer / Case1b_toTaipei))
                    
                    for D in BSTP:
                        car_add = car_transfer / len(BSTP)
                        moto_add = moto_transfer / len(BSTP)
                        rows.append({'origin':int(origin),'dest':int(D),'demand':car_add,'vehType': '1'})
                        rows.append({'origin':int(origin),'dest':int(D),'demand':moto_add,'vehType': '2'})
                        
                elif (origin in BeiShi or origin in SanLu or origin in tamsui) and dest in Neihu:
                    # 修正：先計算轉移量，再減少原始需求
                    car_transfer = car_demand * (0.5 * OBJ_transfer / Case1b_toNeihu)
                    moto_transfer = moto * (0.5 * OBJ_transfer / Case1b_toNeihu)
                    
                    car_demand = car_demand * (1 - (0.5 * OBJ_transfer / Case1b_toNeihu))
                    moto = moto * (1 - (0.5 * OBJ_transfer / Case1b_toNeihu))
                    
                    for D in BSTP:
                        car_add = car_transfer / len(BSTP)
                        moto_add = moto_transfer / len(BSTP)
                        rows.append({'origin':int(origin),'dest':int(D),'demand':car_add,'vehType': '1'})
                        rows.append({'origin':int(origin),'dest':int(D),'demand':moto_add,'vehType': '2'})

            elif case == '全區域':
                if dest in BSTP:
                    car_demand = car_demand * (OBJ_transfer / toBSTP)
                    moto = moto * (OBJ_transfer / toBSTP)
                else:
                    car_demand = car_demand * (1 - (OBJ_transfer / Case2_am))
                    moto = moto * (1 - (OBJ_transfer / Case2_am))
                

            else:
                car_demand = car_demand
                moto = moto

            rows.append({'origin':int(origin),'dest':int(dest),'demand':car_demand,'vehType': '1'})
            rows.append({'origin':int(origin),'dest':int(dest),'demand':moto,'vehType': '2'})
    
    else: # 昏峰
        if origin not in out_of_cordon and dest not in out_of_cordon:
            if case == '士林北投三重':
                if origin in Taipei and (dest in BeiShi or dest in SanLu or dest in tamsui):
                    # 修正：先計算轉移量，再減少原始需求
                    car_transfer = car_demand * (0.5 * OBJ_transfer / Case1b_fromTaipei)
                    moto_transfer = moto * (0.5 * OBJ_transfer / Case1b_fromTaipei)
                    
                    car_demand = car_demand * (1 - (0.5 * OBJ_transfer / Case1b_fromTaipei))
                    moto = moto * (1 - (0.5 * OBJ_transfer / Case1b_fromTaipei))
                    
                    for O in BSTP:
                        car_add = car_transfer / len(BSTP)
                        moto_add = moto_transfer / len(BSTP)
                        rows.append({'origin':int(O),'dest':int(dest),'demand':car_add,'vehType': '1'})
                        rows.append({'origin':int(O),'dest':int(dest),'demand':moto_add,'vehType': '2'})
                        
                elif origin in Neihu and (dest in BeiShi or dest in SanLu or dest in tamsui):
                    car_transfer_total = car_demand * (0.5 * OBJ_transfer / Case1b_fromNeihu)
                    moto_transfer_total = moto * (0.5 * OBJ_transfer / Case1b_fromNeihu)
                
                    car_demand = car_demand * (1 - (0.5 * OBJ_transfer / Case1b_fromNeihu))
                    moto = moto * (1 - (0.5 * OBJ_transfer / Case1b_fromNeihu))
    
                    for O in BSTP:
                        car_add = car_transfer_total / len(BSTP)
                        moto_add = moto_transfer_total / len(BSTP)
                        rows.append({'origin':int(O),'dest':int(dest),'demand':car_add,'vehType': '1'})
                        rows.append({'origin':int(O),'dest':int(dest),'demand':moto_add,'vehType': '2'})
                        
            elif case == '全區域':
                if origin in BSTP:
                    car_demand = car_demand * (OBJ_transfer / fromBSTP)
                    moto = moto * (OBJ_transfer / fromBSTP)
                else:
                    car_demand = car_demand * (1 - (OBJ_transfer / Case2_pm))
                    moto = moto * (1 - (OBJ_transfer / Case2_pm))
                
            else:
                car_demand = car_demand
                moto = moto

            rows.append({'origin':int(origin),'dest':int(dest),'demand':car_demand,'vehType': '1'})
            rows.append({'origin':int(origin),'dest':int(dest),'demand':moto,'vehType': '2'})


print("demand.dat is generated")
df_out = pd.DataFrame(rows)
df_out.to_csv(f'{y}{t}{case}_demand.dat', index=False, sep='\t')

demand.dat is generating...


100%|██████████| 614656/614656 [00:29<00:00, 20880.29it/s]


demand.dat is generated


In [29]:
# DEPRECATED
    # if origin within 691 and dest within 691, then demand *= 1.5
    # if origin <691 and dest <691:
        # demand *= 1.25
    # if origin in 304-691 or dest in 304-691, then demand *= 1.5
    # if origin <= 691 and dest <= 691 and origin >= 304 and dest >= 304:
        # demand *= 1.4
    # if dest within 303, then demand *= 1.5
    # if dest <= 303:
    #     demand *= 1
    # # if origin in tamsui or dest in tamsui:
    # if origin in tamsui:
    #     demand *= 1

        # if case == '士林北投':
        #     if dest in Neihu:
        #         car_distribution = Neihu_tranfer * car_demand / Neihu_carPCU_sum
        #         car_demand = max(car_demand - convert_to_pcu(car_distribution, '1', tp), 0)

        #         moto_distribution = Neihu_tranfer * moto / Neihu_motoPCU_sum
        #         moto = max(moto - convert_to_pcu(moto_distribution, '2', tp), 0)

        #     if dest in BSTP:
        #         car_distribution = BSTP_transfer / len(BSTP) * toNeihu_carPCU[int(origin-1)] / Neihu_carPCU_sum
        #         car_demand = car_demand + convert_to_pcu(car_distribution, '1', tp)

        #         moto_distribution = BSTP_transfer / len(BSTP) * toNeihu_motoPCU[int(origin-1)] / Neihu_motoPCU_sum
        #         moto = moto + convert_to_pcu(moto_distribution, '2', tp)

        # elif case == '北士林北投三重':
        #     if dest in BSTP and origin in BSTPNeighbor:
        #         car_demand = car_demand + convert_to_pcu(BSTPNeighbor_increase, '1', tp)
        #         moto = moto + convert_to_pcu(BSTPNeighbor_increase, '2', tp)

        # elif case == '全區域':
        #     if dest in BSTP:
        #         car_distribution = BSTP_transfer / len(BSTP) * toNeihu_carPCU[int(origin-1)] / Neihu_carPCU_sum
        #         car_demand = car_demand + convert_to_pcu(car_distribution, '1', tp)

        #         moto_distribution = BSTP_transfer / len(BSTP) * toNeihu_motoPCU[int(origin-1)] / Neihu_motoPCU_sum
        #         moto = moto + convert_to_pcu(moto_distribution, '2', tp)